In [1]:
pip install pyhwpx

Note: you may need to restart the kernel to use updated packages.


In [7]:
from pyhwpx import Hwp

# Hwp 인스턴스 생성
hwp = Hwp()

In [ ]:
hwp.Path
hwp.Path

'C:\\Users\\tkdal\\Documents\\ddd.hwp'

In [ ]:
import tkinter as tk
from tkinter import ttk

def on_CellLineMacro():
    """버튼 클릭 이벤트: 드롭다운 값 출력"""
    first_option = dropdown1.get()
    second_option = dropdown2.get()
    #매크로 함수 여기에 추가하기(파라미터로 위에꺼 두개 쓰면될듯)
    print(f"첫행 배경색: {first_option}, 첫행 밑 줄: {second_option}")

def on_ctrl_f1(event):
    """Ctrl+F1 키 입력 이벤트: 버튼 클릭 이벤트 실행"""
    on_CellLineMacro()  # 버튼 클릭 함수 호출

# Tkinter 창 생성
root = tk.Tk()
root.title("표 셀배경 / 선 매크로")
# 테두리 감싸는 Frame 생성
frame = tk.Frame(root, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame.grid(row=0, column=0, padx=10, pady=10)


# 컴포넌트 간 공통 간격
pad_x = 10
pad_y = 10

# region 표탭 - 셀/선 매크로
# 테두리 감싸는 Frame 생성 (레이블, 버튼, 드롭다운 그룹)
frame = tk.Frame(root, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame.grid(row=0, column=0, columnspan=2, padx=20, pady=20)

# 설명 레이블 (사각형 안)
button_description = tk.Label(frame, text="셀/선 매크로", font=("Arial", 12, "bold"))
button_description.grid(row=0, column=0, columnspan=2, padx=pad_x, pady=pad_y)

# 드롭다운 1: 첫행 배경색 (사각형 안)
dropdown1_label = tk.Label(frame, text="첫행 배경색:")
dropdown1_label.grid(row=1, column=0, padx=pad_x, pady=pad_y, sticky="e")

dropdown1 = ttk.Combobox(frame, values=["없음", "회색 255,255,255,15%"])
dropdown1.set("회색 255,255,255,15%")  # 기본값 설정
dropdown1.grid(row=1, column=1, padx=pad_x, pady=pad_y, sticky="w")

# 드롭다운 2: 첫행 밑 줄 (사각형 안)
dropdown2_label = tk.Label(frame, text="첫행 밑 줄:")
dropdown2_label.grid(row=2, column=0, padx=pad_x, pady=pad_y, sticky="e")

dropdown2 = ttk.Combobox(frame, values=["한줄", "두줄"])
dropdown2.set("한줄")  # 기본값 설정
dropdown2.grid(row=2, column=1, padx=pad_x, pady=pad_y, sticky="w")

# 버튼 (사각형 안)
action_button = tk.Button(frame, text="실행 [ctrl+F1]", command=on_CellLineMacro)
action_button.grid(row=3, column=0, columnspan=2, padx=pad_x, pady=pad_y)

# Ctrl+F1 키 이벤트 연결
root.bind("<Control-F1>", on_ctrl_f1)
# endregion
# 창 크기를 UI에 맞게 조정
frame.update_idletasks()  # 내부 위젯 크기 업데이트
root.geometry(f"{frame.winfo_width() + 40}x{frame.winfo_height() + 40}")  # Frame 크기 기준으로 창 크기 설정

# Tkinter 메인 루프 실행
root.mainloop()


In [101]:
import tkinter as tk
from tkinter import ttk


# region tkinter 
# Tkinter 창 생성
root = tk.Tk()
root.title("표 셀배경 / 선 매크로")
#root.geometry("500x400")  # 기본 창 크기 설정

# Notebook(탭 컨트롤러) 생성
notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill="both")
# endregion

# region 표탭 - 셀/선 매크로
def on_CellLineMacro(event=None):
    """버튼 클릭 이벤트: 드롭다운 값 출력"""
    first_option = dropdown1.get()
    second_option = dropdown2.get()
    print(f"첫행 배경색: {first_option}, 첫행 밑 줄: {second_option}")
    #여기에 자동화 함수 추가

def 그림용(event=None):
    print("그림용")
    """그림 표용 세팅"""

def on_CommonlocationMacro(event=None):
    """공통 표 위치 매크로"""
    print("공통매크로")
    #여기 표위치 공통적으로 지정할 것 추가

def handle_1단용(event=None):
    print("1단용 버튼 클릭!")
    on_CommonlocationMacro()
    # 1단용 버튼 동작
def handle_2단_왼_상(event=None):
    print("2단 왼 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 왼 상 동작
def handle_2단_가운_상(event=None):
    print("2단 가운 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 가운 상 동작
def handle_2단_오른_상(event=None):
    print("2단 오른 상 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단 오른 상 동작

def handle_2단_왼_하(event=None):
    print("2단_왼_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_왼_하 동작
def handle_2단_가운_하(event=None):
    print("2단_가운_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_가운_하 동작
def handle_2단_오른_하(event=None):
    print("2단_오른_하 버튼 클릭!")
    on_CommonlocationMacro()
    # 2단_오른_하 동작

def 이전표찾기(event=None):
    print("이전 표 찾기")
def 다음표찾기(event=None):
    print("다음 표 찾기")

def 양옆맞추기(event=None):
    print("양옆맞추기")
    """양옆맞추기"""

# 첫 번째 탭: 표 셀/선 매크로
tab1 = ttk.Frame(notebook)
notebook.add(tab1, text="표 셀/선 매크로")

# '표 위치 매크로' 프레임 추가
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame1.grid(row=0, column=0, columnspan=2, padx=30, pady=10)

# 설명 레이블
position_description = tk.Label(frame1, text="표 이동", font=("Arial", 12, "bold"))
position_description.grid(row=0, column=0, columnspan=4, padx=10, pady=10)

action_button = tk.Button(frame1, text="이전 [1212212]", command=이전표찾기)
action_button.grid(row=1, column=0, padx=10, pady=10)
action_button = tk.Button(frame1, text="다음 [1212122]", command=다음표찾기)
action_button.grid(row=1, column=1,  padx=10, pady=10)
action_button = tk.Button(frame1, text="양옆맞추기", command=양옆맞추기)
action_button.grid(row=2, column=0,columnspan=2, padx=10, pady=10)

# 첫 번째 탭 내용
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
#frame1.pack(expand=True, fill="both", padx=20, pady=20)
frame1.grid(row=1, column=0, columnspan=2, padx=30, pady=10)

button_description = tk.Label(frame1, text="배경/선 매크로", font=("Arial", 12, "bold"))
button_description.grid(row=0, column=0, columnspan=4, padx=10, pady=10)

dropdown1_label = tk.Label(frame1, text="첫행 배경색:")
dropdown1_label.grid(row=1, column=0, padx=10, pady=10, sticky="e")

dropdown1 = ttk.Combobox(frame1, values=["없음", "회색 255,255,255,15%"])
dropdown1.set("회색 255,255,255,15%")
dropdown1.grid(row=1, column=1, padx=10, pady=10, sticky="w")

dropdown2_label = tk.Label(frame1, text="첫행 밑 줄:")
dropdown2_label.grid(row=2, column=0, padx=10, pady=10, sticky="e")

dropdown2 = ttk.Combobox(frame1, values=["한줄", "두줄"])
dropdown2.set("한줄")
dropdown2.grid(row=2, column=1, padx=10, pady=10, sticky="w")

action_button = tk.Button(frame1, text="실행 [ctrl+F1]", command=on_CellLineMacro)
action_button.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

action_button = tk.Button(frame1, text="그림용", command=그림용)
action_button.grid(row=4, column=0,columnspan=2, padx=10, pady=10)


# '표 위치 매크로' 프레임 추가
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame1.grid(row=2, column=0, columnspan=2, padx=30, pady=10)

# 설명 레이블
position_description = tk.Label(frame1, text="표 위치 매크로", font=("Arial", 12, "bold"))
position_description.grid(row=0, column=0, columnspan=4, padx=10, pady=10)

# 오른 단독 버튼
left_button = tk.Button(frame1, text="1단용", command=handle_1단용)
left_button.grid(row=1, column=3, rowspan=2, padx=10, pady=10)


# 6개의 버튼 배열 (1열 3개, 2열 3개)
button1 = tk.Button(frame1, text="2단 왼 상", command=handle_2단_왼_상)
button1.grid(row=1, column=0, padx=10, pady=10)

button2 = tk.Button(frame1, text="2단 가운 상", command=handle_2단_가운_상)
button2.grid(row=1, column=1, padx=10, pady=10)

button3 = tk.Button(frame1, text="2단 오른 상", command=handle_2단_오른_상)
button3.grid(row=1, column=2, padx=10, pady=10)

button4 = tk.Button(frame1, text="2단 왼 하", command=handle_2단_왼_하)
button4.grid(row=2, column=0, padx=10, pady=10)

button5 = tk.Button(frame1, text="2단 가운 하", command=handle_2단_가운_하)
button5.grid(row=2, column=1, padx=10, pady=10)

button6 = tk.Button(frame1, text="2단 오른 하", command=handle_2단_오른_하)
button6.grid(row=2, column=2, padx=10, pady=10)

# '표 위치 매크로' 프레임 추가
frame1 = tk.Frame(tab1, highlightbackground="black", highlightthickness=2, padx=10, pady=10)
frame1.grid(row=0, column=2, columnspan=2, padx=30, pady=10)

# 설명 레이블
position_description = tk.Label(frame1, text="연결 문서", font=("Arial", 12, "bold"))
position_description.grid(row=0, column=0, columnspan=4, padx=10, pady=10)
##여기에 연결된 파일을 표시해주기

# 키 이벤트 연결
root.bind("<Control-F1>", on_CellLineMacro)
# endregion

# region 2번째 탭
# 두 번째 탭: 추가 기능 1
tab2 = ttk.Frame(notebook)
notebook.add(tab2, text="추가 기능 1")

frame2 = tk.Frame(tab2, padx=10, pady=10)
frame2.pack(expand=True, fill="both")
label_tab2 = tk.Label(frame2, text="여기에 두 번째 탭의 내용을 추가하세요.", font=("Arial", 12))
label_tab2.pack(pady=20)
# endregion

# region 3번째 탭
# 세 번째 탭: 추가 기능 2
tab3 = ttk.Frame(notebook)
notebook.add(tab3, text="추가 기능 2")

frame3 = tk.Frame(tab3, padx=10, pady=10)
frame3.pack(expand=True, fill="both")
label_tab3 = tk.Label(frame3, text="여기에 세 번째 탭의 내용을 추가하세요.", font=("Arial", 12))
label_tab3.pack(pady=20)
# endregion

# region UI 출력
# 창 크기를 UI에 맞게 조정
root.update_idletasks()  # 내부 위젯 크기 업데이트
calculated_width = max(200, notebook.winfo_reqwidth() )  # 최소 창 너비 600 설정
calculated_height = max(300, notebook.winfo_reqheight() + 40)  # 최소 창 높이 500 설정
root.geometry(f"{calculated_width}x{calculated_height}")  # 크기 설정

# Tkinter 메인 루프 실행
root.mainloop()
# endregion

